In [ ]:
from scripts.parallel_blocks import *

dirpath = '../data_store/data/sodankyla_full/'

all_ATL03, all_ATL08, failed_ATL03 = track_pairs(dirpath, failed=True)
N = len(all_ATL03)

coords = (26.634154, 67.361833)

i = 35

atl03path, atl08path = all_ATL03[i], all_ATL08[i]

##############################################

A = h5py.File(atl03path, 'r')
if list(A['orbit_info']['sc_orient'])[0] == 1:
    strong = ['gt1r', 'gt2r', 'gt3r']
    weak = ['gt1l', 'gt2l', 'gt3l']
elif list(A['orbit_info']['sc_orient'])[0] == 0:
    strong = ['gt3l', 'gt2l', 'gt1l']
    weak = ['gt3r', 'gt2r', 'gt1r']
else:
    print('Satellite in transition orientation.')
    A.close()
    return 0
tracks = [strong[0], weak[0], strong[1], weak[1], strong[2], weak[2]]
#print(tracks)

# The only purpose of this is to keep the data organised later.
beam_names = [f"Beam {i}" for i in range(1,7)]

A.close()

mid_date = parse_filename_datetime(atl03path)
title_date = datetime_to_title(mid_date)
table_date = datetime_to_date(mid_date)

atl03s = []

for i, gt in enumerate(tracks):
    try:
        atl03 = get_atl03_struct(atl03path, gt, atl08path)
    except (KeyError, ValueError, OSError, IndexError) as e:
        print(f"Failed to open ATL03 file for {foldername} file {file_index}'s beam {i+1}.")
        continue
    # try:
    #     atl08 = get_atl08_struct(atl08path, gt, atl03)
    # except (KeyError, ValueError, OSError) as e:
    #     print(f"Failed to open ATL08 file for {foldername} file {file_index}'s beam {i+1}.")
    #     continue

    atl03.df = atl03.df[(atl03.df['lon_ph'] >= min_lon) & (atl03.df['lon_ph'] <= max_lon) &\
                            (atl03.df['lat_ph'] >= min_lat) & (atl03.df['lat_ph'] <= max_lat)]

    atl03s.append[atl03]

fig = plt.figure(figsize=(10, 12))

ax1 = fig.add_subplot(321)
ax2 = fig.add_subplot(322)
ax3 = fig.add_subplot(323)
ax4 = fig.add_subplot(324)
ax5 = fig.add_subplot(325)
ax6 = fig.add_subplot(326)
axes = [ax1, ax2, ax3, ax4, ax5, ax6]

# Set the figure title
fig.suptitle(title_date + ' - N = ' + str(i), fontsize=16)

